The dataset is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. Now we have 3 pieces of information on hand:
- Enhanced Twitter Archive
- Additional Data via the Twitter API
- Image Predictions File

### Gather

According to the course, I have two materials that can be opened directly with `pandas`
- twitter-archive-enhanced.csv
- image-predictions.tsv

Another one is to get information through the twitter api. I used the program I wrote in `tweet.py`, got a file and saved it as a txt file, and then read it with `pandas`.
- tweet_json.txt

I passed this step very quickly, and I didn't encounter too much difficulty.

### Access

This part actually made me close for a long time. I used the `pandas` method `head` or `info` to browse the information at first, but the data is too much to be effectively browsed in the jupyter notebook. Later, I used Excel to open the file, but a lot of it. The information made me feel powerless at first, I was at a loss.

Later, I asked myself to select only a few lines of information from twitter-archive-enhanced.csv in Excel. I hope to find some problems from it, and I found a literal problem. I recorded it like:
- doggo, floofer, pupper, puppo content is an adjective, can be changed to 0, 1, in addition to hiding a field, you can add these 4 fields to know if multiple adjectives have different results
- retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp. The speculation is to forward the tweet id, which can be checked after the archive table and tweet json data are merged.

Then I used Excel to open tweet_json.txt, but I encountered another problem. Because Excel can't translate the json format smoothly, I have to go back and use code to work. When I open it and it looks like an explosion, I originally wanted to use twitter-archive-enhanced.csv Observing the concept to work, but not very smoothly. But still try to list some items:
- in_reply_to_screen_name, in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str are the same group. When this message is a reply tweet, the related message will be recorded. Then we only consider the original tweet data, so the data is deleted in response to the tweet.
- ...

For the first time, please see [git commit "65d42e"](https://github.com/wrayz/hw-wrangle-and-analyze-data/blob/65d42edbb8a0a9a01d64c88ca9da3570eebcf50c/wrangle_report.ipynb), the content can be written in Chinese Not suitable for English reading, but I want to express one thing from this commit. In the previous Access homework, I was ** guessing the definition of the field based on my own understanding of the content**, this way when doing clean work I found out that this is the wrong way**. The reason is when I question the basis of my speculation? If there is no basis for clean, can such information be used for analysis? The results of these self-answers are **NO**!

To do this, I went to Google to find the twitter api document and quickly found a document [Tweet Objects](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html) It is the details of tweet_json.txt, which means that my Access assignments are coming back, but I have more confidence in this time to have more confidence to evaluate the data.

According to document, I expect to re-accessing tweet_json.txt, then twitter-archive-enhanced.csv, and finally image-predictions.tsv, as follows:

##### Tweet JSON table

**Quality issues**

The first thing to open the data, look at the _missing data_ through `info` and then refer to the document of tweet object, we list these fields and describe them as follows:

- Contributors, coordinates, geo have no data at all, so we don't consider these fields.
- extended_entities records attachment information. Without the data of this field, we can determine that the tweet does not have a picture, so we do not consider these data without pictures.
- in_reply_to_screen_name, in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str are the same group. When this message is a reply tweet, the related message will be recorded. Then we only consider the original tweet data, so the data is deleted in response to the tweet.
- maybe_sensitive, possible_sensitive_appealable Record sensitive information, I think these fields have little to do with this theme, so don't have to keep it.
- place Displays tweet location information, but only one pen, so this field is not retained.
- quoted_status, quoted_status_id, quoted_status_id_str, quoted_status_permalink are the same group. When the message is a quoted tweet, these fields will record the relevant information, just like we don't want the data.
- retweeted_status: Retweets can be distinguished from typical Tweets by the existence of a retweeted_status attribute. So as long as it is forwarded data, we don't want it.

Next look at the other fields:

- display_text_range indicates the range of text, without this field
- entities related information about the original tweets, because there is image prediction to find related image predictions, so this field is not used.
- favorited are all False, basically don't use this field.
- id_str type error, should be string object
- is_quote_status: Indicates whether this is a Quoted Tweet. So as long as it is True we don't want
- lang type error, should be category datatype
- source mixed html tag
- retweeted are False, don't use this field
- truncated are False, don't use this field
- user is the same account @WeRateDogs, this field can also be removed

##### Twitter archive enhanced table

**Quaility issues**

_missing data_
- in_reply_to_status_id, in_reply_to_user_id field information is also available in Tweet JSON, we can merge the table and then check if the data is a reply message, if it is deleted
- retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp. The speculation is to forward the tweet id, which can be checked after the archive table and tweet json data are merged.
- expanded_urls, using Excel to view the content found that there are many duplicate urls, currently do not know why

_other_
- Erroneous datatypes: timestamp type should be changed to datetime
- source is the same source as the Tweet JSON, with the html tag language.

**Tidiness issues**
- doggo, floofer, pupper, puppo content is an adjective, can be changed to 0, 1, in addition to hiding a field, you can add these 4 fields to know if multiple adjectives have different results
- rating_numerator, rating_denominator also hides a field message rating_numerator/rating_denominator can get a rough ratio

##### Image prediction table

**Quality issues**
- There are 324 data to determine which dog breed

**Tidiness issues**
- There are three predictions in the data, which hides one field and the final result
- This information can be combined with the Twitter archive enhanced table to combine the best predicted results (dog breeds).

### Clean

Executing a clean job, the original plan is to clean up one by one according to the Accessing column, but I noticed
1. Accessing matters must be summarized again
2. During cleaning, I still need to re-do the Accessing step and become cleaning-accessing-cleaning when necessary. The purpose of this is just to make sure that I am doing this without much error.

Below I will list the following items according to different information:

##### Tweet json table

- Remove unwanted fields, contributors, coordinates, geo, possible_sensitive, possible_sensitive_appealable, place, display_text_range, entities, favorited, retweeted, truncated, user
- Remove extended_entities missing data
- Delete in_reply_to_screen_name, in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str data with value
- Delete data with is_quote_status True
- Delete retweeted_status valued data
- Check the rest of the data and remove the fields without data
- Remove the is_quote_status, extended_entities field
- Corrected id_str type as str datatype
- Corrected lang type as category datatype
- trim source content don't html info

#### Twitter archive enhanced table
- Merge Twitter archive enhanced table and Tweet json table.
- Use info to check the information after the merge. If there is no data in the field, you can remove it directly.
- Calculate rating_numerator / rating_denominator and put it in a new field rating_ratio
- doggo, floofer, pupper, puppo content is an adjective, can be changed to 0, 1, in addition to hiding a field, you can add these 4 fields to know if multiple adjectives have different results

#### Image prediction table
- Delete data with no variety (a total of 324)
- Find the dog breed according to the priority of the forecast results and put it in the new open field.
- merge with Twitter archive enhanced table

## Storing Data

Finally I finally got a clean piece of information that I can use for analysis!

I saved it to the twitter_archive_master.csv file and saved it to SqlLite. For me, this step is very easy.

In [1]:
import pandas as pd
df = pd.read_csv('twitter_archive_master.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1663 entries, 0 to 1662
Data columns (total 15 columns):
tweet_id          1663 non-null int64
text              1663 non-null object
expanded_urls     1663 non-null object
name              1663 non-null object
doggo             1663 non-null object
floofer           1663 non-null object
pupper            1663 non-null object
puppo             1663 non-null object
favorite_count    1663 non-null int64
lang              1663 non-null object
retweet_count     1663 non-null int64
source            1663 non-null object
created_at        1663 non-null object
rating_ratio      1663 non-null float64
breed             1663 non-null object
dtypes: float64(1), int64(3), object(11)
memory usage: 195.0+ KB


In [2]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'wrangle_report.ipynb'])

0